In [1]:
import re
import time
import urllib
import requests
import numpy as np
from bs4 import BeautifulSoup
from openpyxl import Workbook

In [2]:
#Some User Agents
hds=['Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6',
'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.12 Safari/535.11',
 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Trident/6.0)']

In [3]:
def getHtml(url):
    req = urllib.request.Request(url, headers={'User-Agent': hds[1%len(hds)],'cookie': cookie})
    html = urllib.request.urlopen(req).read()
    return html
 
def saveHtml(file_name, file_content):
 # 注意windows文件命名的禁用符，比如 /
 with open(file_name.replace('/', '_') + ".html", "wb") as f:
  # 写文件用bytes而不是str，所以要转码
  f.write(file_content)

In [4]:
def my_spider(cookie):
    page_num=1;
    book_list=[]
    try_times=0
    err = 0

    while(1):
        url=URL+str(page_num)
        time.sleep(np.random.rand()*5)

        #Last Version
        try:
            req = urllib.request.Request(url, headers={'User-Agent': hds[page_num%len(hds)],'cookie': cookie})
            source_code = urllib.request.urlopen(req).read()
            plain_text=str(source_code, 'utf-8')
            print("请求成功")
        except:
            print("请求出错！")
            continue

        soup = BeautifulSoup(plain_text)
        list_soup = soup.findAll('tr')

        print("新增了 {} 条数据".format(len(list_soup)))
        try_times+=1;
        if list_soup==None and try_times<3:
            continue
        elif list_soup==None or len(list_soup)<=1:
            err+=1
            if(err>5):
                break # Break when no informatoin got after 200 times requesting


        for info in list_soup:
            aim = info.find('a', attrs={"href" : re.compile(r'^problem')})
            label = info.find('a', {'class': 'label'})
            if aim !=None and label!=None:
                link = BASE_URL + aim.get('href').replace("\\'","")
                content = str(aim.string)
                source = str(label.string)
                html = getHtml(link)
                saveHtml(source+content, html)
                time.sleep(np.random.rand()*3)
                book_list.append([link, content, source])
                try_times=0 #set 0 when got valid information
        page_num+=1
        print('Downloading Information From Page %d' % page_num)
    return book_list


In [ ]:
BASE_URL = 'https://oj.neauacm.cn/'
URL = 'https://oj.neauacm.cn/problemset.php?page='
cookie = 'resolveIDs=0; order_dir_list_by=1A; PHPSESSID=2f9f83jjfrl45dpn1oeodf18s0; lastlang=undefined'
book_list = my_spider(cookie)

请求成功
新增了 64 条数据
请求成功
新增了 103 条数据
请求成功
新增了 103 条数据


In [ ]:
book_list

In [ ]:
len(book_list)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
arr = np.array(book_list)

In [ ]:
data = pd.DataFrame({"link": arr[:,0], "name": arr[:,1]})

In [ ]:
data

In [ ]:
data.to_excel("题目.xlsx")